In [2]:
import os
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image

import matplotlib.pyplot as plt

In [3]:
path_train = '../basep/output/train/'
path_test = '../basep/output/test/'

In [4]:
# Clase personalizada para cargar el dataset
class FingerprintDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith('.png')]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name)
        angle = float(self.image_files[idx].split('_r')[-1].split('.png')[0])
        
        if self.transform:
            image = self.transform(image)
            
        return image, angle


In [5]:
# Definir transformaciones y cargar el dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

train_dataset = FingerprintDataset(path_train, transform=transform)
test_dataset = FingerprintDataset(path_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
# Crear el modelo ViT
model = timm.create_model("vit_base_patch16_224", pretrained=True, num_classes=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [7]:
# Entrenar el modelo
epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [8]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

TOLERANCE = 5  # Ajusta según lo que consideres adecuado

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_predictions_train = 0
    total_predictions_train = 0
    
    for images, angles in train_loader:
        images, angles = images.to(device), angles.to(device).float().unsqueeze(1)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, angles)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Calcular "precisión" dentro de la tolerancia para entrenamiento
        correct_predictions_train += torch.sum(torch.abs(outputs - angles) < TOLERANCE).item()
        total_predictions_train += angles.size(0)
    
    train_loss = total_loss/len(train_loader)
    train_losses.append(train_loss)
    train_accuracy = correct_predictions_train / total_predictions_train
    train_accuracies.append(train_accuracy)
    
    # Validación
    model.eval()
    total_val_loss = 0
    correct_predictions_val = 0
    total_predictions_val = 0
    
    with torch.no_grad():
        for images, angles in test_loader:
            images, angles = images.to(device), angles.to(device).float().unsqueeze(1)
            
            outputs = model(images)
            loss = criterion(outputs, angles)
            
            total_val_loss += loss.item()
            
            # Calcular "precisión" dentro de la tolerancia para validación
            correct_predictions_val += torch.sum(torch.abs(outputs - angles) < TOLERANCE).item()
            total_predictions_val += angles.size(0)
    
    val_loss = total_val_loss/len(test_loader)
    val_losses.append(val_loss)
    val_accuracy = correct_predictions_val / total_predictions_val
    val_accuracies.append(val_accuracy)
    
    print(f"Epoch {epoch+1}/{epochs}:\n\tTraining Loss: {train_loss}, Validation Loss: {val_loss}\n\t Training Accuracy: {train_accuracy}, Validation Accuracy: {val_accuracy}")



Epoch 1/100:
	Training Loss: 85.53983935371774, Validation Loss: 82.84413334580718
	 Training Accuracy: 0.32773217034376606, Validation Accuracy: 0.30271934325295025
Epoch 2/100:
	Training Loss: 80.72162129449063, Validation Loss: 81.98325660580494
	 Training Accuracy: 0.3297845048742945, Validation Accuracy: 0.30631092868137505
Epoch 3/100:
	Training Loss: 80.25705242156982, Validation Loss: 82.73641773911773
	 Training Accuracy: 0.3268342739866598, Validation Accuracy: 0.29502308876346844
Epoch 4/100:
	Training Loss: 82.86789373491631, Validation Loss: 82.54226421919026
	 Training Accuracy: 0.3304258594150847, Validation Accuracy: 0.30631092868137505
Epoch 5/100:
	Training Loss: 81.09941957817703, Validation Loss: 82.97304584940926
	 Training Accuracy: 0.3297845048742945, Validation Accuracy: 0.31144176500769627
Epoch 6/100:
	Training Loss: 81.28997743325155, Validation Loss: 82.42019234328974
	 Training Accuracy: 0.32901487942534635, Validation Accuracy: 0.30631092868137505
Epoch 7/

In [1]:
# Graficar las pérdidas de entrenamiento y validación
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss', color='tab:red')
plt.plot(val_losses, label='Validation Loss', color='tab:orange')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss over Epochs')
plt.grid(True)
plt.tight_layout()
plt.show()

# Graficar la "precisión" dentro de la tolerancia para entrenamiento y validación
plt.figure(figsize=(10, 6))
plt.plot(train_accuracies, label='Training Accuracy', color='tab:blue')
plt.plot(val_accuracies, label='Validation Accuracy', color='tab:cyan')
plt.xlabel('Epoch')
plt.ylabel('Accuracy within Tolerance')
plt.legend()
plt.title('Training and Validation Accuracy within Tolerance over Epochs')
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined